# Build your First Deep Learning Neural Network Model using Keras in Python
https://medium.com/@pushkarmandot/build-your-first-deep-learning-neural-network-model-using-keras-in-python-a90b5864116d

In [1]:
import theano
import tensorflow
import keras

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


## Step 1: 
Importing data. Pandas DataFrame gives massive functionality to work on data thus, here we are using pandas to import data.

In [2]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')

In [4]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
dataset.shape

(10000, 14)

## Step 2: 
Create matrix of features and matrix of target variable. In this case we are excluding column 1 & 2 as those are ‘row_number’ and ‘customerid’ which are not useful in our analysis. Column 14, ‘Exited’ is our Target Variable

In [6]:
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

In [7]:
X

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [8]:
y

array([1, 0, 1, ..., 1, 1, 0], dtype=int64)

## Step 3:
Let’s make analysis simpler by encoding string variables. Country has string labels such as “France, Spain, Germany” while Gender has “Male, Female”. We have to encode this strings into numeric and we can simply do it using pandas but here I am introducing new library called `ScikitLearn` which is strongest machine learning library in python. We will use `LabelEncoder`. As the name suggests, whenever we pass a variable to this function, this function will automatically encode different labels in that column with values between 0 to n_classes-1.

In [9]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])

In [10]:
X

array([[619, 0, 0, ..., 1, 1, 101348.88],
       [608, 2, 0, ..., 0, 1, 112542.58],
       [502, 0, 0, ..., 1, 0, 113931.57],
       ...,
       [709, 0, 0, ..., 0, 1, 42085.58],
       [772, 1, 1, ..., 1, 0, 92888.52],
       [792, 0, 0, ..., 1, 0, 38190.78]], dtype=object)

## Step 4: 
How to create dummy variable in python? We will use the same ScikitLearn library but this time we will use another function called as `OneHotEncoder`, yeah it is seriously hot. We just need to pass the column number and whoosh your dummy variable is created.

NOTE: OneHotEncoder() `categorical_features=` was deprecated. Used ColumnTransformer() instead, per:  
https://stackoverflow.com/questions/54345667/onehotencoder-categorical-features-deprecated-how-to-transform-specific-column

In [11]:
from sklearn.compose import ColumnTransformer

In [12]:
ct = ColumnTransformer([('Country', OneHotEncoder(), [1])], remainder='passthrough')

In [13]:
X = ct.fit_transform(X)

In [14]:
X[:5]

array([[1.0, 0.0, 0.0, 619, 0, 42, 2, 0.0, 1, 1, 1, 101348.88],
       [0.0, 0.0, 1.0, 608, 0, 41, 1, 83807.86, 1, 0, 1, 112542.58],
       [1.0, 0.0, 0.0, 502, 0, 42, 8, 159660.8, 3, 1, 0, 113931.57],
       [1.0, 0.0, 0.0, 699, 0, 39, 1, 0.0, 2, 0, 0, 93826.63],
       [0.0, 0.0, 1.0, 850, 0, 43, 2, 125510.82, 1, 1, 1, 79084.1]],
      dtype=object)

In [15]:
X[:, 1:][:5]

array([[0.0, 0.0, 619, 0, 42, 2, 0.0, 1, 1, 1, 101348.88],
       [0.0, 1.0, 608, 0, 41, 1, 83807.86, 1, 0, 1, 112542.58],
       [0.0, 0.0, 502, 0, 42, 8, 159660.8, 3, 1, 0, 113931.57],
       [0.0, 0.0, 699, 0, 39, 1, 0.0, 2, 0, 0, 93826.63],
       [0.0, 1.0, 850, 0, 43, 2, 125510.82, 1, 1, 1, 79084.1]],
      dtype=object)

In [16]:
X.shape

(10000, 12)

In [17]:
X[:, 1:][:5].shape

(5, 11)

Okay, whatevs:

In [18]:
X = X[:, 1:]

## Step 5:
We will make use of ScikitLearn’s `train_test_split` function to divide our data. Roughly people keep 80:20, 75:25, 60:40 as their train test split ratio. Here we are keeping it as 80:20.

In [19]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

## Step 6: 
`StandardScaler` is available in ScikitLearn. In the following code we are fitting and transforming StandardScaler method on train data. We have to standardize our scaling so we will use the same fitted method to transform/scale test data.

In [20]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [21]:
X_train[:5]

array([[-0.57792762,  1.72744428,  0.50196358,  0.90934104,  0.77397459,
        -0.34952914, -1.21975497, -0.9165696 ,  0.65192707, -1.02943297,
         0.38241231],
       [ 1.73032049, -0.57888987,  0.76049847,  0.90934104,  0.58397515,
         1.03269188,  0.65194003, -0.9165696 , -1.53391391, -1.02943297,
         1.28105488],
       [-0.57792762, -0.57888987,  0.50196358,  0.90934104, -0.08102289,
         1.03269188,  0.93840411, -0.9165696 ,  0.65192707, -1.02943297,
        -1.71346336],
       [ 1.73032049, -0.57888987,  0.52264637,  0.90934104, -0.27102232,
         1.37824714,  0.23000423,  0.80307318,  0.65192707, -1.02943297,
         1.29504677],
       [-0.57792762,  1.72744428, -0.07715457,  0.90934104, -1.98101727,
         0.68713663, -1.21975497, -0.9165696 , -1.53391391,  0.97140856,
        -0.69864061]])

# Neural Network time!

## Step 7: 
Importing required Modules. We need **Sequential module** for initializing NN and **dense module** to add Hidden Layers.

In [22]:
# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense

## Step 8: 
I am giving the name of model as **Classifier** as our business problem is the classification of customer churn. In the last step, I mentioned that we will use Sequential module for initialization so here it is:

In [23]:
#Initializing Neural Network
classifier = Sequential()

## Step 9: 
Adding multiple hidden layer will take bit effort. We will add **hidden layers** one by one using dense function. In the below code you will see a lot of arguments. Don’t worry I will explain them all.

NOTE: I changed `output_dim=` to `units=` and changed `init=` to `kernel_initializer=`  
See here: https://keras.io/api/layers/core_layers/dense/

In [24]:
# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

Here we are using **Sigmoid** function in our output layer as we want binary result from output layer but if the number of categories in output layer is more than 2 then use **SoftMax** function.

## Step 10: 
Till now we have added multiple layers to our classifier now let’s compile them which can be done using compile method. Arguments added in final compilation will control whole neural network so be careful on this step. I will briefly explain arguments.

In [25]:
# Compiling Neural Network
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Since our dependent variable is binary, we will have to use logarithmic loss function called **‘binary_crossentropy’**, if our dependent variable has more than 2 categories in output then use **‘categorical_crossentropy’**.  
We want to improve performance of our neural network based on accuracy so add metrics as **'accuracy'**.

**Congratulations, you have build your first Deep Learning Neural Network model.**

## Step 11: 
We will now train our model on training data but still one thing is remaining. We use fit method to the fit our model In previous some steps I said that we will be optimizing our weights to improve model efficiency so when are we updating our weights? **Batch size** is used to specify the number of observation after which you want to update weight. **Epoch** is nothing but the total number of iterations. Choosing the value of batch size and epoch is trial and error there is no specific rule for that.  
NOTE: I replaced `nb_epoch=` with `epochs=`

In [26]:
%%time 
# Fitting our model 
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
800/800 [==============================] - 1s 771us/step - loss: 0.4739 - accuracy: 0.7980
Epoch 2/100
800/800 [==============================] - 1s 728us/step - loss: 0.4225 - accuracy: 0.7981
Epoch 3/100
800/800 [==============================] - 1s 706us/step - loss: 0.4167 - accuracy: 0.8052
Epoch 4/100
800/800 [==============================] - 1s 708us/step - loss: 0.4134 - accuracy: 0.8240
Epoch 5/100
800/800 [==============================] - 1s 709us/step - loss: 0.4112 - accuracy: 0.8284
Epoch 6/100
800/800 [==============================] - 1s 712us/step - loss: 0.4097 - accuracy: 0.8305
Epoch 7/100
800/800 [==============================] - 1s 715us/step - loss: 0.4087 - accuracy: 0.8319
Epoch 8/100
800/800 [==============================] - 1s 708us/step - loss: 0.4071 - accuracy: 0.8331
Epoch 9/100
800/800 [==============================] - 1s 706us/step - loss: 0.4060 - accuracy: 0.8332
Epoch 10/100
800/800 [==============================] - 1s 751us/step - l

## Step 12: 
**Predicting** the test set result. The prediction result will give you probability of the customer leaving the company. We will convert that probability into binary 0 and 1.

In [27]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

## Step 13: 
This is the final step where we are evaluating our model performance. We already have original results and thus we can build **confusion matrix** to check the accuracy of model.

In [28]:
# Creating the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [29]:
cm

array([[1509,   69],
       [ 217,  205]], dtype=int64)

In [30]:
cm.sum()

2000

In [31]:
cm[0][0]

1509

In [32]:
accuracy = (cm[0][0] + cm[1][1]) / cm.sum()
round(accuracy * 100, 2)

85.7